In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [43]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [60]:
class VisionTransformer(nn.Module):
    def __init__(self):
        super(VisionTransformer, self).__init__()
        self.embeddingLayer = nn.Conv2d(3,768, 16, 16)
        self.positionalEncoding = PositionalEncoding(3072)
        self.encoderLayers = []
        for i in range(12):
            self.encoderLayers.append(nn.TransformerEncoderLayer(3072, 12, 3072))

    def forward(self, x):
        x = self.embeddingLayer(x)
                
        n, c, w, h = x.shape
        print(x.shape)
        x = torch.reshape(x, [n, h * w, c])
        
        print(x.shape)
        x = self.positionalEncoding(x)
        for encoderLayer in self.encoderLayers:
            print("called")
            x = encoderLayer(x)
        return x

In [61]:
test = torch.rand(3000, 3, 32, 32)
vision_transformer = VisionTransformer()
res = vision_transformer(test)
res.shape

torch.Size([3000, 768, 2, 2])
torch.Size([3000, 4, 768])


RuntimeError: The size of tensor a (768) must match the size of tensor b (3072) at non-singleton dimension 2

In [26]:
testConv = nn.Conv2d(3, 15, 16, 16)
test = torch.rand(1,3,17,17)
l = testConv(test)
print(l.shape)

torch.Size([1, 15, 1, 1])
